In [1]:
import pandas as pd
import seaborn as sns
import glob
import os
import subprocess

In [2]:
# Load in sample sheet from Zhoufeng
df = pd.read_excel('GSP_Sample_Sheet.xlsx')
df.tail()

,No.,Sample Code,Country,City,Sample amount received,Sampling date,Shipping date,Sample received at DTU,Sample condition upon receipt,ml spinned,Sample appearance - INTERNAL DTU COMMENT,DTU tube labeled,DTU Freezer,"DTU Placement (→,↓)",Sample Filtered
44,45,1024,United Arab Emirates,Dubai (Al Aweer stp),1L,2021-05-31 00:00:00,11/07/2021,14-07-2021 hnni,Thawed,500 ml,Pellet 1 gik tabt da den blev blandet sammen m...,1024,Rainbow,"Large 11, small 13",1 of 2
45,46,1020,United Kingdom,Camborne,1L,2021-04-06 11:31:00,29/06/2021,06-07-2021 bsrl,Thawed,500 ml,NaN,1020,Rainbow,"Large 11, small 13",1 of 2
46,47,963,"USA, WA",Seattle (West Point Treatment Plant),1L,2021-04-19 00:00:00,Unknown,20-05-2021 bsrl,Thawed with a big lump of ice,500 ml,NaN,963,Rainbow,"Large 12, small 15",1 of 2
47,48,985,"USA, VA","Christiansburg, VA",1L,2021-05-07 00:00:00,Unknown,2021-06-10 00:00:00,Thawed with lump of ice,500 ml,NaN,985,Rainbow,"Large 12, small 14",1 of 2
48,49,969,"USA, WI","South Milwaukee, WI",2x1L,21/05/2021 11:00,24/05/2021,26-05-2021 chsv,Frozen,500 ml,NaN,969,Rainbow,"Large 12, small 15",1 of 2


In [3]:
# Load in metadata (Supplementary Data 1)
meta = pd.read_excel('GS3_Supplementary_Data.xlsx', sheet_name='SD1 Sample metadata')
meta['name_meta'] = meta['complete_name'].str.split(r'_S\d{1,3}_L\d{3}').apply(lambda x: x[0])

# Translate the sample numbers to complete_name
meta['sample_code'] = meta['name_meta'].str.extract(r"_(\d{3,4})$").fillna(0).astype(int)
df = df.merge(meta[['sample_code', 'complete_name', 'genepid']], left_on='Sample Code', right_on='sample_code')

In [4]:
target_file = 'GSP_samples_data.xlsx'

writer = pd.ExcelWriter(target_file, mode='a', if_sheet_exists='replace')

df.to_excel(writer, sheet_name='sample_metadata', index=None)

# PanRes metadata

In [5]:
# PanRes metadata can be received from https://zenodo.org/records/13885013
panres_meta = pd.read_csv('https://zenodo.org/records/13885013/files/panres_annotations.tsv', sep='\t', index_col=0)
panres_meta['gene'] = panres_meta['gene'].str.replace('_v1.0.2', '')
panres_meta_wide = panres_meta.pivot_table(index='gene', columns='variable', values='value', aggfunc=';'.join) 

In [6]:
# Extract gene names for ResFinder genes
resfinder_genes = panres_meta.query("variable == 'database' & value == 'resfinder'")['gene'].unique().tolist()
resfinder_fa = panres_meta_wide.loc[resfinder_genes, ['fa_header', 'class', 'gene_length']]
resfinder_fa['fa_header'] = resfinder_fa['fa_header'].apply(lambda x: [v for v in x.split(';') if v.startswith('resfinder')][0].replace('resfinder|', ''))
resfinder_fa['gene_length'] = resfinder_fa['gene_length'].astype(int)
resfinder_fa.to_excel(writer, sheet_name='resfinder_metadata', index=None)


In [7]:
# Extract names for biocide genes
biocide_genes = panres_meta.query("variable == 'resistance_type' & value == 'biocide'")['gene'].unique().tolist()
biocide_fa = panres_meta_wide.loc[biocide_genes, ['fa_header', 'class', 'gene_length']]
biocide_fa['fa_header'] = biocide_fa['fa_header'].apply(lambda x: [v for v in x.split(';') if v.startswith('megares')][0].replace('megares|', ''))

biocide_fa['gene_name'] = biocide_fa['fa_header'].str.extract(r"\|(\w+)$")
biocide_fa['compound'] = biocide_fa['fa_header'].apply(lambda x: x.split('|')[2].replace('_resistance', ''))
biocide_fa['gene_length'] = biocide_fa['gene_length'].astype(int)

biocide_fa.to_excel(writer, sheet_name='biocide_metadata', index=None)


# Find count data

In [8]:
panres_counts = pd.read_csv('https://zenodo.org/records/14652833/files/panres_counts.csv', index_col=0)
panres_counts.head()

,refSequence,readCount,fragmentCount,mapScoreSum,refCoveredPositions,refConsensusSum,bpTotal,depthVariance,nucHighDepthVariance,depthMax,...,fragmentCountAln,sample_name,sample_name2,gene,cluster_representative_98,gene_length,fragmentCountAln_adj,fragmentCountAln_adj_kb,complete_name,genepid
0,pan_1,1825,1744,1142,531,531,1142,1.303267,0,4,...,5,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,DTU_2021_1011464_1_MG_EC_QU_871,pan_1,pan_1,825,0.006061,6.060606,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,1011464.0
1,pan_8,384,309,17753,684,684,17759,263.016820,0,50,...,64,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,DTU_2021_1011464_1_MG_EC_QU_871,pan_8,pan_8,1392,0.045977,45.977011,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,1011464.0
2,pan_9,3910,2769,307890,1920,1920,308553,1271.387270,0,217,...,1154,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,DTU_2021_1011464_1_MG_EC_QU_871,pan_9,pan_13734,1920,0.601042,601.041667,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,1011464.0
3,pan_11,1338,1189,33049,831,831,33184,93.338419,0,57,...,162,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,DTU_2021_1011464_1_MG_EC_QU_871,pan_11,pan_9365,831,0.194946,194.945848,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,1011464.0
4,pan_17,505,279,71452,800,800,71524,3184.781592,0,193,...,261,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,DTU_2021_1011464_1_MG_EC_QU_871,pan_17,pan_1617,801,0.325843,325.842697,DTU_2021_1011464_1_MG_EC_QU_871_S25_L004,1011464.0


In [9]:
total_fragments = pd.read_csv('https://zenodo.org/records/14652833/files/tot_counts.csv', index_col=0).drop_duplicates()
total_fragments = total_fragments.loc[total_fragments['complete_name'].isin(df['complete_name'])]
total_fragments = total_fragments.drop(columns=['sample_name', 'name', 'bindCol'])
total_fragments.head()

,total_fragments,complete_name
2018,42143463,DTU_2022_1024401_1_MG_DZ_BC_907_S0_L001
2019,41533472,DTU_2022_1024402_1_MG_ZA_PR_908_S0_L001
2020,39364491,DTU_2022_1024403_1_MG_KR_DA_909_S0_L001
2022,42366317,DTU_2022_1024405_1_MG_CZ_PR_911_S0_L001
2023,38967740,DTU_2022_1024406_1_MG_AT_VI_912_S0_L001


In [10]:
resfinder_mapstat = panres_counts.merge(resfinder_fa.drop(columns='gene_length'), left_on=['refSequence'], right_on=['gene'])
biocide_mapstat = panres_counts.merge(biocide_fa.drop(columns='gene_length'), left_on='refSequence', right_on='gene')

resfinder_mapstat = resfinder_mapstat.merge(df, on='complete_name')
biocide_mapstat = biocide_mapstat.merge(df, on='complete_name')

In [11]:
totals = total_fragments.merge(
    resfinder_mapstat.groupby(['complete_name']).agg({'fragmentCountAln_adj_kb': 'sum', 'refSequence': 'nunique'}).reset_index(),
    on='complete_name'
).merge(
    biocide_mapstat.groupby(['complete_name']).agg({'fragmentCountAln_adj_kb': 'sum', 'refSequence': 'nunique'}).reset_index(),
    on='complete_name'
).rename(
    columns={
        'fragmentCountAln_adj_kb_x': 'total_resfinder_counts',
        'fragmentCountAln_adj_kb_y': 'total_biocide_counts',
        'refSequence_x': 'n_resfinder_genes',
        'refSequence_y': 'n_biocide_genes',
    }
)

In [12]:
totals[['complete_name', 'n_resfinder_genes', 'n_biocide_genes']].to_excel(writer, sheet_name='total_genes_found', index=None)

## Extract total counts

The mOTUs total counts can be retrieved from https://zenodo.org/records/14652833/files/motus_counts.zip
```{bash}
wget https://zenodo.org/records/14652833/files/motus_counts.zip
unzip motus_counts.zip
cp motus_counts/motus_agg_pad/kingdom_motus_pad_agg_counts.csv .
rm -rf motus_counts
rm motus_counts.zip
```


In [13]:
motu_kingdom_counts = pd.read_csv('kingdom_motus_pad_agg_counts.csv')
bacterial_counts = df[['complete_name', 'genepid']].merge(motu_kingdom_counts[['genepid', 'Bacteria']], on='genepid').drop(columns='genepid').rename(columns={'Bacteria': 'total_bacteria_counts'})

In [14]:
totals.merge(bacterial_counts, on='complete_name')[['complete_name', 'total_fragments', 'total_bacteria_counts', 'total_resfinder_counts', 'total_biocide_counts']].to_excel(writer, sheet_name='total_counts', index=None)

## ResFinder counts

In [15]:
resfinder_mapstat['class_split'] = resfinder_mapstat['class'].str.split(';')

resfinder_gene_matches = resfinder_mapstat.drop(columns='class').explode('class_split').pivot_table(
    index='complete_name',
    columns='class_split',
    values='gene',
    aggfunc=lambda x: len(list(x))
)

resfinder_gene_matches.to_excel(writer, sheet_name='resfinder_class_gene_matches')

resfinder_mapstat.explode('class_split').pivot_table(
    index='complete_name',
    columns = 'class',
    values = 'fragmentCountAln_adj_kb',
    aggfunc='sum',
    fill_value=0
).to_excel('resfinder_class_counts.xlsx')

## Biocide counts

In [16]:
biocide_mapstat.pivot_table(
    index='complete_name',
    columns='compound',
    values='gene',
    aggfunc=lambda x: len(list(x))
).to_excel(writer, sheet_name='biocide_class_gene_matches')

biocide_mapstat.pivot_table(
    index='complete_name',
    columns = 'compound',
    values = 'fragmentCountAln_adj_kb',
    aggfunc='sum',
    fill_value=0
).to_excel('biocide_gene_counts.xlsx')

In [17]:
writer.close()